In [ ]:
%pip install pandas yfinance tqdm

In [143]:
import pandas as pd
import yfinance as yf
from tqdm import tqdm

In [144]:
# Specify the input file path for macOS
input_file = '/Users/christopher/Desktop/Seminar-Advanced-Finance/stocks.csv'

# Specify the output file path for macOS
output_file = '/Users/christopher/Desktop/Seminar-Advanced-Finance/stocks_data.csv'

# Read the CSV file
stocks_df = pd.read_csv(input_file)

In [145]:
# Filter rows to only include tickers in the 'Financials' and 'Industrials' sectors
stocks_df = stocks_df[stocks_df['GICS Sector'].isin(['Financials', 'Industrials'])]

# List to accumulate data frames for each ticker
final_df = pd.DataFrame()

In [146]:
# Loop (iteration) over each row/ticker
for idx, row in tqdm(stocks_df.iterrows(), 
                     total=stocks_df.shape[0],
                     desc="Downloading stock data"):
    
    # Extract the ticker symbol & GICS sector from the row
    ticker = row['Symbol']       
    gics_sector = row['GICS Sector']

    # Download data from yfinance
    data = yf.download(ticker, start="2000-01-01", end="2025-01-01", progress=False, auto_adjust=False)

    # Add 'Ticker' and 'GICS Sector' as normal columns
    data["Ticker"] = ticker
    data["GICS Sector"] = gics_sector

    # 3) Reset index so we have a "Date" column (instead of an index)
    data.reset_index(inplace=True)

    data.columns = data.columns.droplevel(1)

    data = data[['Date', 'Adj Close', 'Ticker', 'GICS Sector']]

    final_df = pd.concat([final_df, data], ignore_index=True)

In [148]:
final_df.to_csv(output_file, index=False)